In [1]:
import pandas as pd
import numpy as np
import math
import os
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# ===================== 1. Basic configuration + memory optimization function =====================
DATA_DIR = 'D:/m5-forecasting-accuracy/'


def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtype

        if pd.api.types.is_numeric_dtype(col_type):
            c_min = df[col].min()
            c_max = df[col].max()
            if pd.api.types.is_integer_dtype(col_type):
                if c_min >= 0 and c_max <= 255:
                    df[col] = df[col].astype(np.uint8)
                elif c_max <= 65535:
                    df[col] = df[col].astype(np.uint16)
                elif c_max <= 4294967295:
                    df[col] = df[col].astype(np.uint32)
                else:
                    df[col] = df[col].astype(np.int32)
            elif pd.api.types.is_float_dtype(col_type):
                df[col] = df[col].astype(np.float32)
        elif col_type == object:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024 ** 2
    print(f'Memory usage from {start_mem:.2f} MB drop to {end_mem:.2f} MB')
    return df

FILES = {
    'sales_val': 'sales_train_validation.csv',
    'sales_eval': 'sales_train_evaluation.csv',
    'calendar': 'calendar.csv',
    'sell_prices': 'sell_prices.csv',
    'sample_sub': 'sample_submission.csv'
}
for file_name, file_path in FILES.items():
    full_path = os.path.join(DATA_DIR, file_path)
    if not os.path.exists(full_path):
        raise FileNotFoundError(f"File does not exist: {full_path}")

# ===================== 2. Read and preprocess the calendar table =====================
calendar = pd.read_csv(os.path.join(DATA_DIR, FILES['calendar']), encoding='latin1')
calendar['day_number'] = calendar['d'].str.extract(r'd_(\d+)').astype(int)
calendar = calendar[['d', 'day_number', 'wm_yr_wk', 'wday', 'month', 'year']]
calendar = reduce_mem_usage(calendar)

# ===================== 3. Read submission sample (fix item/store extraction logic) =====================
sample_sub = pd.read_csv(os.path.join(DATA_DIR, FILES['sample_sub']), encoding='latin1')
# Fix 1: More robust dataset extraction (avoid index out of bounds after splitting)
sample_sub['dataset'] = sample_sub['id'].apply(lambda x: x.split('_')[-1] if '_' in x else '')


# Fix 2: Item_id/store_id extraction (compatible with all ID formats)
def extract_item_store(id_str):
    # Example ID: HOBBIES_1_001_CA_1_validation → remove the last field after splitting
    parts = id_str.split('_')
    if len(parts) >= 4:
        # Take the first n-3 parts to form item_id, last two parts to form store_id
        item_id = '_'.join(parts[:-3])
        store_id = '_'.join(parts[-3:-1])
    else:
        item_id = ''
        store_id = ''
    return item_id, store_id


# Batch extraction
sample_sub[['item_id', 'store_id']] = sample_sub['id'].apply(
    lambda x: pd.Series(extract_item_store(x))
)
sample_sub = reduce_mem_usage(sample_sub)
print(
    f"Total submission samples: {len(sample_sub)}, extracted {sample_sub['item_id'].nunique()} unique items, {sample_sub['store_id'].nunique()} unique stores")

# ===================== 4. Read sell prices table =====================
sell_prices = pd.read_csv(os.path.join(DATA_DIR, FILES['sell_prices']), encoding='latin1')
sell_prices = reduce_mem_usage(sell_prices)
print(
    f"Total sell price records: {len(sell_prices)}, covering {sell_prices['item_id'].nunique()} unique items, {sell_prices['store_id'].nunique()} unique stores")


# ===================== 5. Chunk processing function (with detailed logs) =====================
def process_sales_chunk(chunk, is_validation, chunk_idx):
    print(f"\n===== Processing chunk {chunk_idx} (validation={is_validation}) =====")
    # Reshape wide sales table to long format
    d_cols = [col for col in chunk.columns if col.startswith('d_')]
    if not d_cols:
        print("No date columns in chunk, skipped")
        return None
    df_melt = chunk.melt(
        id_vars=['id', 'item_id', 'store_id'],
        value_vars=d_cols,
        var_name='d',
        value_name='sales'
    )
    df_melt['dataset'] = 'validation' if is_validation else 'evaluation'
    df_melt = reduce_mem_usage(df_melt)
    print(f"Chunk reshaped to long format, total rows: {len(df_melt)}")

    # Merge calendar features
    df_melt = df_melt.merge(calendar, on='d', how='left')
    # Generate cyclic features
    df_melt['wday_sin'] = np.sin(2 * math.pi * df_melt['wday'] / 7).astype(np.float32)
    df_melt['wday_cos'] = np.cos(2 * math.pi * df_melt['wday'] / 7).astype(np.float32)
    df_melt['month_sin'] = np.sin(2 * math.pi * df_melt['month'] / 12).astype(np.float32)
    df_melt['month_cos'] = np.cos(2 * math.pi * df_melt['month'] / 12).astype(np.float32)

    # Merge sell price features
    df_melt = df_melt.merge(sell_prices, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')
    df_melt['sell_price'] = df_melt['sell_price'].fillna(0).astype(np.float32)
    df_melt = reduce_mem_usage(df_melt)

    # Split training set
    train_data = df_melt[df_melt['day_number'] <= 1913].copy()
    print(f"Training set rows in chunk: {len(train_data)}")
    if len(train_data) == 0:
        print("Training set is empty, skipped")
        return None

    # Construct test set
    chunk_item_stores = df_melt[['item_id', 'store_id']].drop_duplicates()
    print(f"Chunk contains {len(chunk_item_stores)} unique item-store combinations")

    test_ids = sample_sub[
        (sample_sub['item_id'].isin(chunk_item_stores['item_id'])) &
        (sample_sub['store_id'].isin(chunk_item_stores['store_id'])) &
        (sample_sub['dataset'] == ('validation' if is_validation else 'evaluation'))  # New: Match dataset type
        ]
    print(f"Number of matched test IDs in chunk: {len(test_ids)}")
    if len(test_ids) == 0:
        print("No matched test IDs, skipped")
        return None

    # Generate test set date rows
    pred_days = {
        'validation': [f'd_{i}' for i in range(1914, 1942)],
        'evaluation': [f'd_{i}' for i in range(1942, 1970)]
    }
    test_data_list = []
    for _, row in test_ids.iterrows():
        for d in pred_days[row['dataset']]:
            test_data_list.append({
                'id': row['id'],
                'item_id': row['item_id'],
                'store_id': row['store_id'],
                'd': d
            })
    test_data = pd.DataFrame(test_data_list)
    test_data = test_data.merge(calendar, on='d', how='left')
    # Generate cyclic features for test set
    test_data['wday_sin'] = np.sin(2 * math.pi * test_data['wday'] / 7).astype(np.float32)
    test_data['wday_cos'] = np.cos(2 * math.pi * test_data['wday'] / 7).astype(np.float32)
    test_data['month_sin'] = np.sin(2 * math.pi * test_data['month'] / 12).astype(np.float32)
    test_data['month_cos'] = np.cos(2 * math.pi * test_data['month'] / 12).astype(np.float32)
    # Merge sell price features
    test_data = test_data.merge(sell_prices, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')
    test_data['sell_price'] = test_data['sell_price'].fillna(0).astype(np.float32)
    test_data = reduce_mem_usage(test_data)

    # Build model
    X_train = train_data[['wday_sin', 'wday_cos', 'month_sin', 'month_cos', 'year', 'sell_price']]
    y_train = train_data['sales']
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), ['wday_sin', 'wday_cos', 'month_sin', 'month_cos', 'year', 'sell_price'])]
    )
    model = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', Ridge(alpha=1.0))])
    model.fit(X_train, y_train)

    # Predict
    X_test = test_data[['wday_sin', 'wday_cos', 'month_sin', 'month_cos', 'year', 'sell_price']]
    y_pred = model.predict(X_test)
    y_pred = np.maximum(y_pred, 0).astype(np.float32)
    test_data['sales_pred'] = y_pred

    # Organize results
    test_data['F'] = test_data.groupby('id')['day_number'].rank(method='first').astype(int)
    test_data['F_col'] = 'F' + test_data['F'].astype(str)
    chunk_sub = test_data.pivot_table(
        index='id', columns='F_col', values='sales_pred', aggfunc='first'
    ).reset_index()
    chunk_sub = chunk_sub[['id'] + [f'F{i}' for i in range(1, 29)]]
    print(f"Chunk generated {len(chunk_sub)} prediction results")
    return chunk_sub


# ===================== 6. Execute chunk processing =====================
chunk_size = 10000
submission_list = []

# Process validation sales data
print("========== Start processing validation chunks ==========")
for i, chunk in enumerate(
        pd.read_csv(os.path.join(DATA_DIR, FILES['sales_val']), chunksize=chunk_size, encoding='latin1')):
    chunk = reduce_mem_usage(chunk)
    chunk_sub = process_sales_chunk(chunk, is_validation=True, chunk_idx=i + 1)
    if chunk_sub is not None:
        submission_list.append(chunk_sub)

# Process evaluation sales data (Optional: Comment first to validate validation only)
print("\n========== Start processing evaluation chunks ==========")
for i, chunk in enumerate(pd.read_csv(os.path.join(DATA_DIR, FILES['sales_eval']), chunksize=chunk_size, encoding='latin1')):

    chunk = reduce_mem_usage(chunk)
    chunk_sub = process_sales_chunk(chunk, is_validation=False, chunk_idx=i+1)
    if chunk_sub is not None:
         submission_list.append(chunk_sub)

# ===================== 7. Merge results (add deduplication logic) =====================
print(f"\n========== Merge results ==========")
print(f"Number of valid prediction chunks: {len(submission_list)}")

# Fallback: Generate all-zero submission if no results
if not submission_list:
    print("No valid prediction results, generate all-zero submission file")
    submission = sample_sub[['id']].copy()
    for f in [f'F{i}' for i in range(1, 29)]:
        submission[f] = 0.0
else:
    submission = pd.concat(submission_list, ignore_index=True)
    # Key modification: Deduplicate by ID, keep the last row for each ID
    submission = submission.drop_duplicates(subset='id', keep='last').reset_index(drop=True)
    print(f"Remaining unique IDs after deduplication: {len(submission)}")

    # Supplement missing IDs
    all_ids = sample_sub['id'].tolist()
    missing_ids = [id_ for id_ in all_ids if id_ not in submission['id'].tolist()]
    if missing_ids:
        missing_df = pd.DataFrame({'id': missing_ids})
        for f in [f'F{i}' for i in range(1, 29)]:
            missing_df[f] = 0.0
        submission = pd.concat([submission, missing_df], ignore_index=True)

    # Sort by original ID order
    submission = submission.set_index('id').loc[all_ids].reset_index()

# Save submission file
submission_path = os.path.join(DATA_DIR, 'M5_ridge_submission.csv')
submission.to_csv(submission_path, index=False, encoding='latin1')
print(f"Final submission file saved to: {submission_path}")
print(f"Total rows in submission file: {len(submission)}")

Memory usage from 0.09 MB drop to 0.10 MB
Memory usage from 14.89 MB drop to 4.66 MB
Total submission samples: 60980, extracted 3049 unique items, 10 unique stores
Memory usage from 208.77 MB drop to 58.80 MB
Total sell price records: 6841121, covering 3049 unique items, 10 unique stores
========== Start processing validation chunks ==========
Memory usage from 146.41 MB drop to 21.00 MB

===== Processing chunk 1 (validation=True) =====
Memory usage from 420.02 MB drop to 182.93 MB
Chunk reshaped to long format, total rows: 19130000
Memory usage from 930.85 MB drop to 693.76 MB
Training set rows in chunk: 19130000
Chunk contains 10000 unique item-store combinations
Number of matched test IDs in chunk: 12196
Memory usage from 19.54 MB drop to 11.51 MB


C:\Users\Tho'r\AppData\Local\Temp\ipykernel_19468\121281277.py:188: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  test_data['F'] = test_data.groupby('id')['day_number'].rank(method='first').astype(int)
C:\Users\Tho'r\AppData\Local\Temp\ipykernel_19468\121281277.py:190: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  chunk_sub = test_data.pivot_table(


Chunk generated 12196 prediction results
Memory usage from 146.41 MB drop to 19.15 MB

===== Processing chunk 2 (validation=True) =====
Memory usage from 420.02 MB drop to 182.93 MB
Chunk reshaped to long format, total rows: 19130000
Memory usage from 930.85 MB drop to 693.76 MB
Training set rows in chunk: 19130000
Chunk contains 10000 unique item-store combinations
Number of matched test IDs in chunk: 12196
Memory usage from 19.54 MB drop to 11.51 MB


C:\Users\Tho'r\AppData\Local\Temp\ipykernel_19468\121281277.py:188: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  test_data['F'] = test_data.groupby('id')['day_number'].rank(method='first').astype(int)
C:\Users\Tho'r\AppData\Local\Temp\ipykernel_19468\121281277.py:190: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  chunk_sub = test_data.pivot_table(


Chunk generated 12196 prediction results
Memory usage from 146.41 MB drop to 19.04 MB

===== Processing chunk 3 (validation=True) =====
Memory usage from 420.02 MB drop to 182.93 MB
Chunk reshaped to long format, total rows: 19130000
Memory usage from 930.85 MB drop to 693.76 MB
Training set rows in chunk: 19130000
Chunk contains 10000 unique item-store combinations
Number of matched test IDs in chunk: 12196
Memory usage from 19.54 MB drop to 11.51 MB


...

Chunk generated 12196 prediction results
Memory usage from 148.54 MB drop to 19.31 MB

===== Processing chunk 3 (validation=False) =====
Memory usage from 426.16 MB drop to 185.60 MB
Chunk reshaped to long format, total rows: 19410000
Memory usage from 944.47 MB drop to 703.90 MB
Training set rows in chunk: 19130000
Chunk contains 10000 unique item-store combinations
Number of matched test IDs in chunk: 12196
Memory usage from 19.54 MB drop to 11.51 MB


C:\Users\Tho'r\AppData\Local\Temp\ipykernel_19468\121281277.py:188: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  test_data['F'] = test_data.groupby('id')['day_number'].rank(method='first').astype(int)
C:\Users\Tho'r\AppData\Local\Temp\ipykernel_19468\121281277.py:190: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  chunk_sub = test_data.pivot_table(


Chunk generated 12196 prediction results
Memory usage from 7.28 MB drop to 0.95 MB

===== Processing chunk 4 (validation=False) =====
Memory usage from 20.90 MB drop to 9.19 MB
Chunk reshaped to long format, total rows: 951090
Memory usage from 51.72 MB drop to 34.58 MB
Training set rows in chunk: 937370
Chunk contains 490 unique item-store combinations
Number of matched test IDs in chunk: 490
Memory usage from 0.79 MB drop to 0.49 MB


C:\Users\Tho'r\AppData\Local\Temp\ipykernel_19468\121281277.py:188: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  test_data['F'] = test_data.groupby('id')['day_number'].rank(method='first').astype(int)
C:\Users\Tho'r\AppData\Local\Temp\ipykernel_19468\121281277.py:190: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  chunk_sub = test_data.pivot_table(


Chunk generated 490 prediction results

========== Merge results ==========
Number of valid prediction chunks: 8
Remaining unique IDs after deduplication: 60980
Final submission file saved to: D:/m5-forecasting-accuracy/M5_ridge_submission.csv
Total rows in submission file: 60980
